In [22]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import pandas as pd
import time
import pendulum

load_dotenv()

API_KEY = os.getenv("YT_DATA_API")
youtube = build(serviceName="youtube", version="v3", developerKey=API_KEY)
VIDEO_BATCH_SIZE = 50
CHANNEL_BATCH_SIZE = 50

data_path='../data'

In [23]:
videos_df=pd.read_csv(f'{data_path}/videos.csv')
#comment for production
videos_df=videos_df[:1]
print(videos_df.shape)

(1, 8)


In [24]:
# single video res json dump
# {
#   "kind": "youtube#videoListResponse",
#   "etag": "HmRhfk46xUPBIkmr1vMuymkAjnA",
#   "items": [
#     {
#       "kind": "youtube#video",
#       "etag": "m2vOYc7R1BsMXRuHH_F8qSMdQrk",
#       "id": "kR626gjg0MQ",
#       "snippet": {
#         "publishedAt": "2024-07-04T21:50:14Z",
#         "channelId": "UCHMnsOPDgrjWvjHXZp-xvqg",
#         "title": "Zenless Zone Zero Beginner's Guide - 20 Tips to Progress Smoothly!",
#         "description": "Zenless Zone Zero is here! Here are 20 tips and mistakes to avoid for beginners and gacha veterans alike!\n\nJoin the Puff club for some Pufferfish emojis! Thank you for your support \u2764\ufe0f\nhttps://www.youtube.com/channel/UCHMnsOPDgrjWvjHXZp-xvqg/join\n\n0:00 | Game Overview\n1:10 | Account Progression\n1:47 | Side Quests\n2:30 | Survey Data\n3:01 | Stamina Info\n3:50 | Daily Stuff\n4:05 | More Progression\n4:30 | Refresh?\n4:55 | Challenge Mode\n5:17 | Overworld Friendship\n5:45 | Thoroughly Explore\n6:13 | Teambuilding Faction\n7:06 | Team Archetypes\n7:40 | Endgame Speculation\n7:55 | Investment Priority\n8:17 | FREE PULLS via leveling\n9:03 | Gacha Breakdown\n10:40 | Bangboo Banner\n11:17 | WEngine Bait\n11:53 | TIP !IMPORTANT\n13:00 | Don't Sweat It\n\n#zenlesszonezero #zzz",
#         "thumbnails": {
#           "default": {
#             "url": "https://i.ytimg.com/vi/kR626gjg0MQ/default.jpg",
#             "width": 120,
#             "height": 90
#           },
#           "medium": {
#             "url": "https://i.ytimg.com/vi/kR626gjg0MQ/mqdefault.jpg",
#             "width": 320,
#             "height": 180
#           },
#           "high": {
#             "url": "https://i.ytimg.com/vi/kR626gjg0MQ/hqdefault.jpg",
#             "width": 480,
#             "height": 360
#           },
#           "standard": {
#             "url": "https://i.ytimg.com/vi/kR626gjg0MQ/sddefault.jpg",
#             "width": 640,
#             "height": 480
#           },
#           "maxres": {
#             "url": "https://i.ytimg.com/vi/kR626gjg0MQ/maxresdefault.jpg",
#             "width": 1280,
#             "height": 720
#           }
#         },
#         "channelTitle": "IWinToLose Gaming",
#         "tags": [
#           "zenless zone zero",
#           "zenless zone zero gameplay",
#           "zenless zone zero characters",
#           "zenless zone zero guide",
#           "zenless zone zero review",
#           "zenless zone zero beginner guide",
#           "zenless zone zero release date",
#           "zzz guide",
#           "zzz gameplay",
#           "zenless zone zero starter guide",
#           "zzz review",
#           "zenless zone zero f2p",
#           "zenless zone zero beginner tips",
#           "zenless zone zero combat guide",
#           "everything you need to know in zenless zone zero",
#           "zenless zone zero starter tips"
#         ],
#         "categoryId": "20",
#         "liveBroadcastContent": "none",
#         "defaultLanguage": "en",
#         "localized": {
#           "title": "Zenless Zone Zero Beginner's Guide - 20 Tips to Progress Smoothly!",
#           "description": "Zenless Zone Zero is here! Here are 20 tips and mistakes to avoid for beginners and gacha veterans alike!\n\nJoin the Puff club for some Pufferfish emojis! Thank you for your support \u2764\ufe0f\nhttps://www.youtube.com/channel/UCHMnsOPDgrjWvjHXZp-xvqg/join\n\n0:00 | Game Overview\n1:10 | Account Progression\n1:47 | Side Quests\n2:30 | Survey Data\n3:01 | Stamina Info\n3:50 | Daily Stuff\n4:05 | More Progression\n4:30 | Refresh?\n4:55 | Challenge Mode\n5:17 | Overworld Friendship\n5:45 | Thoroughly Explore\n6:13 | Teambuilding Faction\n7:06 | Team Archetypes\n7:40 | Endgame Speculation\n7:55 | Investment Priority\n8:17 | FREE PULLS via leveling\n9:03 | Gacha Breakdown\n10:40 | Bangboo Banner\n11:17 | WEngine Bait\n11:53 | TIP !IMPORTANT\n13:00 | Don't Sweat It\n\n#zenlesszonezero #zzz"
#         },
#         "defaultAudioLanguage": "en"
#       },
#       "contentDetails": {
#         "duration": "PT13M30S",
#         "dimension": "2d",
#         "definition": "hd",
#         "caption": "false",
#         "licensedContent": true,
#         "contentRating": {},
#         "projection": "rectangular"
#       },
#       "statistics": {
#         "viewCount": "452725",
#         "likeCount": "13185",
#         "favoriteCount": "0",
#         "commentCount": "770"
#       }
#     }
#   ],
#   "pageInfo": {
#     "totalResults": 1,
#     "resultsPerPage": 1
#   }
# }

In [25]:
def chunk(arr,batch_size):
    res=[]
    i=0
    n=len(arr)
    while i<n:
        res.append(arr[i:min(n,i+batch_size)])
        i+=batch_size
    return res
    
import re

ISO_8601_DURATION_REGEX = re.compile(
    r'P'                                  # starts with 'P'
    r'(?:(\d+)D)?'                        # days
    r'(?:T'                               # time component
    r'(?:(\d+)H)?'                        # hours
    r'(?:(\d+)M)?'                        # minutes
    r'(?:(\d+)S)?'                        # seconds
    r')?'                                 # time component optional
)

def parse_iso8601_duration(duration: str) -> int:

    if not duration or not duration.startswith("P"):
        raise ValueError(f"Invalid ISO 8601 duration: {duration}")

    match = ISO_8601_DURATION_REGEX.fullmatch(duration)
    if not match:
        raise ValueError(f"Invalid ISO 8601 duration: {duration}")

    days, hours, minutes, seconds = match.groups(default="0")

    total_seconds = (
        int(days) * 86400 +
        int(hours) * 3600 +
        int(minutes) * 60 +
        int(seconds)
    )

    return total_seconds
    
# import json      
def fetch_video_metadata(video_ids: list[str]) -> list[dict]:
    results = []

    for batch in chunk(video_ids, VIDEO_BATCH_SIZE):
        res = youtube.videos().list(
            part="snippet,statistics,contentDetails",
            id=",".join(batch)
        ).execute()
        # print(json.dumps(res,indent=2))

        for item in res.get("items", []):
            results.append({
                "video_id": item["id"],
                "channel_id": item["snippet"]["channelId"],
                "title": item["snippet"]["title"],
                "description": item["snippet"].get("description", ""),
                "tags": item["snippet"].get("tags", []),
                "published_at": pendulum.parse(item["snippet"]["publishedAt"]),
                "duration_seconds":parse_iso8601_duration(item["contentDetails"]["duration"]), #https://developers.google.com/youtube/v3/docs/videos#contentDetails.duration
                "views": int(item["statistics"].get("viewCount", 0)),
                "likes": int(item["statistics"].get("likeCount", 0)),
                "comments": int(item["statistics"].get("commentCount", 0))
            })

    return results
    
video_ids=list(videos_df['video_id'])
print(video_ids)
video_metadata = fetch_video_metadata(video_ids)
print(video_metadata)

['kR626gjg0MQ']
[{'video_id': 'kR626gjg0MQ', 'channel_id': 'UCHMnsOPDgrjWvjHXZp-xvqg', 'title': "Zenless Zone Zero Beginner's Guide - 20 Tips to Progress Smoothly!", 'description': "Zenless Zone Zero is here! Here are 20 tips and mistakes to avoid for beginners and gacha veterans alike!\n\nJoin the Puff club for some Pufferfish emojis! Thank you for your support ❤️\nhttps://www.youtube.com/channel/UCHMnsOPDgrjWvjHXZp-xvqg/join\n\n0:00 | Game Overview\n1:10 | Account Progression\n1:47 | Side Quests\n2:30 | Survey Data\n3:01 | Stamina Info\n3:50 | Daily Stuff\n4:05 | More Progression\n4:30 | Refresh?\n4:55 | Challenge Mode\n5:17 | Overworld Friendship\n5:45 | Thoroughly Explore\n6:13 | Teambuilding Faction\n7:06 | Team Archetypes\n7:40 | Endgame Speculation\n7:55 | Investment Priority\n8:17 | FREE PULLS via leveling\n9:03 | Gacha Breakdown\n10:40 | Bangboo Banner\n11:17 | WEngine Bait\n11:53 | TIP !IMPORTANT\n13:00 | Don't Sweat It\n\n#zenlesszonezero #zzz", 'tags': ['zenless zone zero',

In [26]:
# single channel res
# {
#   "kind": "youtube#channelListResponse",
#   "etag": "HAxXcK3hZ5HJgr6AUl4oe8LAY64",
#   "pageInfo": {
#     "totalResults": 1,
#     "resultsPerPage": 5
#   },
#   "items": [
#     {
#       "kind": "youtube#channel",
#       "etag": "IFl-mH37dW9yZWbItClRsC0wGpQ",
#       "id": "UCHMnsOPDgrjWvjHXZp-xvqg",
#       "snippet": {
#         "title": "IWinToLose Gaming",
#         "description": "Business inquiries - iwintolose@mythictalent.com\n\nA dedicated gamer making analysis, showcase, and whatever other videos for the game(s) I play. Currently working on Genshin Impact and Honkai Star Rail.\n\nAffiliated with Starroad: business@cdstarroad.com\n",
#         "customUrl": "@iwintolose",
#         "publishedAt": "2016-03-09T17:24:29Z",
#         "thumbnails": {
#           "default": {
#             "url": "https://yt3.ggpht.com/ytc/AIdro_lcQk8T1X4kbKO0zvLYqcusiYXDfE2AJbgGncDQ1Qmcsk8=s88-c-k-c0x00ffffff-no-rj",
#             "width": 88,
#             "height": 88
#           },
#           "medium": {
#             "url": "https://yt3.ggpht.com/ytc/AIdro_lcQk8T1X4kbKO0zvLYqcusiYXDfE2AJbgGncDQ1Qmcsk8=s240-c-k-c0x00ffffff-no-rj",
#             "width": 240,
#             "height": 240
#           },
#           "high": {
#             "url": "https://yt3.ggpht.com/ytc/AIdro_lcQk8T1X4kbKO0zvLYqcusiYXDfE2AJbgGncDQ1Qmcsk8=s800-c-k-c0x00ffffff-no-rj",
#             "width": 800,
#             "height": 800
#           }
#         },
#         "localized": {
#           "title": "IWinToLose Gaming",
#           "description": "Business inquiries - iwintolose@mythictalent.com\n\nA dedicated gamer making analysis, showcase, and whatever other videos for the game(s) I play. Currently working on Genshin Impact and Honkai Star Rail.\n\nAffiliated with Starroad: business@cdstarroad.com\n"
#         }
#       },
#       "statistics": {
#         "viewCount": "175191432",
#         "subscriberCount": "497000",
#         "hiddenSubscriberCount": false,
#         "videoCount": "782"
#       }
#     }
#   ]
# }

In [27]:
# import json
def fetch_channel_metadata(channel_ids: list[str]) -> dict:
    channels = {}

    for batch in chunk(channel_ids, CHANNEL_BATCH_SIZE):
        res = youtube.channels().list(
            part="snippet,statistics",
            id=",".join(batch)
        ).execute()
        # print(json.dumps(res,indent=2))
        
        for item in res.get("items", []):
            channels[item["id"]] = {
                "channel_id": item["id"],
                "channel_name": item["snippet"]["title"],
                "subscribers": int(item["statistics"].get("subscriberCount", 0)),
                "video_count": int(item["statistics"].get("videoCount", 0)),
                "created_at": pendulum.parse(item["snippet"]["publishedAt"])
            }

    return channels
channel_ids=list(set(videos_df['channel_id']))
print(channel_ids)
channel_metadata = fetch_channel_metadata(channel_ids)
print(channel_metadata)

['UCHMnsOPDgrjWvjHXZp-xvqg']
{'UCHMnsOPDgrjWvjHXZp-xvqg': {'channel_id': 'UCHMnsOPDgrjWvjHXZp-xvqg', 'channel_name': 'IWinToLose Gaming', 'subscribers': 497000, 'video_count': 785, 'created_at': DateTime(2016, 3, 9, 17, 24, 29, tzinfo=Timezone('UTC'))}}


In [28]:
def compute_engagement(video: dict) -> float:
    if video["views"] == 0:
        return 0.0
    return (video["likes"] + video["comments"]) / video["views"]

In [85]:
import ast
agents=pd.read_csv(f'{data_path}/dim_agent.csv')
agents['aliases']=agents['aliases'].apply(ast.literal_eval)

def extract_characters(title: str, description: str, tags: list[str]) -> list[dict]:
    text = f"{title} {description} {' '.join(tags)}".lower()

    matched = []

    for _,agent in agents.iterrows():
        if any(alias.lower() in text for alias in agent["aliases"]):
            matched.append(agent["name"])

    weight = 1 / len(matched) if matched else 0

    return [
        {"name": name, "weight": weight}
        for name in matched
    ]

In [86]:
def build_enriched_video_record(video: dict, channel: dict) -> dict:
    engagement_rate = compute_engagement(video)

    characters = extract_characters(
        title=video["title"],
        description=video["description"],
        tags=video["tags"]
    )

    return {
        **video,
        "channel_name": channel["channel_name"],
        "subscribers": channel["subscribers"],
        "engagement_rate": engagement_rate,
        "publish_hour": video["published_at"].hour,
        "day_of_week": video["published_at"].weekday(),
        "characters": characters,
        "run_date": pendulum.now()
    }

In [87]:
enriched_videos = []
for video in video_metadata:
    channel = channel_metadata.get(video["channel_id"])
    if not channel:
        continue

    enriched = build_enriched_video_record(
        video=video,
        channel=channel
    )
    enriched_videos.append(enriched)
# persist_enriched_videos(enriched_videos)
# persist_channel_snapshots(channel_metadata.values(), run_date)

In [89]:
result_df=pd.DataFrame(enriched_videos)

,video_id,channel_id,title,description,tags,published_at,duration_seconds,views,likes,comments,channel_name,subscribers,engagement_rate,publish_hour,day_of_week,characters,run_date
0,kR626gjg0MQ,UCHMnsOPDgrjWvjHXZp-xvqg,Zenless Zone Zero Beginner's Guide - 20 Tips t...,Zenless Zone Zero is here! Here are 20 tips an...,"[zenless zone zero, zenless zone zero gameplay...",2024-07-04 21:50:14+00:00,810,453221,13198,771,IWinToLose Gaming,497000,0.030822,21,3,[],2026-01-24 17:13:01.519231+05:30


In [ ]:
if not os.path.exists(data_path):
    os.mkdir(data_path)
df.to_csv(f"{data_path}/enriched_videos.csv", index=False)